In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew, norm
from scipy.optimize import minimize
from numba import njit
from typing import Tuple, Optional
from enum import Enum
import qis
from dataclasses import fields, replace, asdict
from datetime import datetime, timedelta
import scipy.stats as ss
import copy
import scipy

from scipy.interpolate import splrep, BSpline
from numba.typed import List

# analytics
import sys
sys.path.insert(0,'../../') # just for jupyter  notebook
from stochvolmodels.pricers.hawkes_jd_pricer import HawkesJDParams, HawkesJDPricer, hawkesjd_chain_pricer, unpack_and_transform_pars_for_measure_change, unpack_pars

import tensorflow.experimental.numpy as tnp
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from stochvolmodels.data.test_option_chain import get_btc_test_chain_data, get_gld_test_chain_data_6m, get_sqqq_test_chain_data, get_spy_test_chain_data
from stochvolmodels.utils.funcs import to_flat_np_array, set_time_grid, timer, set_seed, transform_to_tfcomplex128, transform_from_tfcomplex128_to_np, slice_option_chain
from stochvolmodels.data.option_chain import OptionChain

import os
from stochvolmodels.pricers.core.bsm_pricer import infer_bsm_implied_vol, compute_bsm_price
import warnings
from stochvolmodels.MLE_estimator import hawkes_jd
from stochvolmodels.MLE_estimator import hawkes_jd_weekday

import pickle

warnings.filterwarnings('ignore')

2023-11-06 09:56:30.538452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def slice_option_chain(option_chain, ids_i):
    option_chain = copy.copy(option_chain)
    replacements = dict()
    for field in fields(option_chain):
        field_name = field.name
        option_chain_value = getattr(option_chain, field_name)
        try:
            replacements[field_name] = [option_chain_value[i] for i in ids_i]
        except:
            replacements[field_name] = option_chain_value
            
    return OptionChain(**replacements)

In [3]:
with open('BTC_option_chain.pickle', 'rb') as f:
    BTC_option_chain_dict = pickle.load(f)
    
P_params = pd.read_csv('P_params.csv', index_col=0)

In [5]:
dates_arr = pd.Series(BTC_option_chain_dict.keys())

year = 2020

if year == 2019:
    _id = pd.to_datetime(dates_arr) > '2018-12-31'
    _id *= pd.to_datetime(dates_arr) <= '2019-12-31'

elif year == 2020:
    _id = pd.to_datetime(dates_arr) > '2019-12-31'
    _id *= pd.to_datetime(dates_arr) <= '2020-12-31'

elif year == 2021:
    _id = pd.to_datetime(dates_arr) > '2020-12-31'
    _id *= pd.to_datetime(dates_arr) <= '2021-12-31'

elif year == 2022:
    _id = pd.to_datetime(dates_arr) > '2021-12-31'
    _id *= pd.to_datetime(dates_arr) <= '2022-12-31'
    
elif year == 2023:
    _id = pd.to_datetime(dates_arr) > '2022-12-31'


dates_arr = list(dates_arr.loc[_id])

In [6]:
theta_p = P_params.loc[:,'theta_p'].iloc[0]
theta_m = P_params.loc[:,'theta_m'].iloc[0]
kappa_p = P_params.loc[:,'kappa_p'].iloc[0]
kappa_m = P_params.loc[:,'kappa_m'].iloc[0]
beta11  = P_params.loc[:,'beta11'].iloc[0]
beta12  = P_params.loc[:,'beta12'].iloc[0]
beta21  = P_params.loc[:,'beta21'].iloc[0]
beta22  = P_params.loc[:,'beta22'].iloc[0]
eta_p   = P_params.loc[:,'eta_p'].iloc[0]
eta_m   = P_params.loc[:,'eta_m'].iloc[0]
nu_p    = P_params.loc[:,'nu_p'].iloc[0]
nu_m    = P_params.loc[:,'nu_m'].iloc[0]

mu = 0
sigma = P_params.sigma.iloc[0]


In [7]:
def smooth_option_chain(option_chain, N_K=41):
    new_K_arr_ttms = []
    new_ivs_ttms = []
    new_optiontypes_ttms = []

    for i in range(len(option_chain.strikes_ttms)):
        K_arr = option_chain.strikes_ttms[i]
        mid_ivs = (option_chain.bid_ivs[i]+option_chain.ask_ivs[i])/2
        bid_ask_spread = np.max(option_chain.bid_ivs[i]+option_chain.ask_ivs[i])
        
        min_K = option_chain.strikes_ttms[i].min()
        max_K = option_chain.strikes_ttms[i].max()
        new_K_arr = np.linspace(min_K, max_K, N_K)
        try:
            tck = splrep(K_arr, mid_ivs, s=bid_ask_spread)
            new_ivs = BSpline(*tck)(new_K_arr)
        except:
            tck = splrep(K_arr, mid_ivs, k=2)
            new_ivs = BSpline(*tck)(new_K_arr)
        new_optiontypes = np.array(['P']*len(new_K_arr), dtype='<U1')
        C_id = new_K_arr < option_chain.forwards[i]
        new_optiontypes[C_id] = 'C'
        
        new_K_arr_ttms.append(new_K_arr)
        new_ivs_ttms.append(new_ivs)
        
        new_optiontypes_ttms.append(new_optiontypes)
        
    new_option_chain = OptionChain(ids=option_chain.ids,
        ttms=option_chain.ttms,
        ticker='BTC',
        forwards=option_chain.forwards,
        strikes_ttms=new_K_arr_ttms,
        optiontypes_ttms=new_optiontypes_ttms,
        discfactors=option_chain.discfactors,
        bid_ivs=new_ivs_ttms,
        ask_ivs=new_ivs_ttms)

    return new_option_chain
    
    
# plt.scatter(K_arr, option_chain.bid_ivs[i], color='green')
# plt.scatter(K_arr, option_chain.ask_ivs[i], color='red')

# plt.scatter(K_arr, mid_ivs)
# plt.scatter(new_K_arr, new_ivs)
    

In [23]:
Q_results_1m = dict()


In [24]:
pricer = HawkesJDPricer(M=2**10, x_max=10, n_steps_per_ttm=100)

bounds = ( (0.12, None), (None, None), (None, None) )
method = 'powell'
options = {'ftol':1e-7, 'tol':1e-7, 'maxiter':1e8}
errors_weights = (1, 1e-8, 1e-8, 0, 0)

chi0_arr = [(-2,-4)]

mu = 0
sigma = 0.2

MAPE_thres = 0.0
for date in dates_arr:
# for date in ['2021-11-15']:
    print('-----------', date, '-----------')
    if (date in Q_results_1m):
        if (Q_results_1m[date]['MAPE'] > MAPE_thres) or (np.isnan(Q_results_1m[date]['MAPE'])):
            pass
        else:
            continue
    else:
        pass
    lambda_p = P_params.loc[date,:].lambda_p_right
    lambda_m = P_params.loc[date,:].lambda_m_right
    is_pos_jump = P_params.loc[date,:].jump_sizes > 0 
    is_neg_jump = P_params.loc[date,:].jump_sizes < 0 
    
    params = HawkesJDParams(mu=mu, sigma=sigma,
                            eta_p=eta_p, nu_p=nu_p, eta_m=eta_m, nu_m=nu_m,
                            theta_p=theta_p, kappa_p=kappa_p, theta_m=theta_m, kappa_m=kappa_m,
                            beta11=beta11, beta21=beta21, beta12=beta12, beta22=beta22,
                            lambda_p=lambda_p, lambda_m=lambda_m)
    
    params = transform_to_tfcomplex128(params)
    
    _id = BTC_option_chain_dict[date].ttms * 365 <= 45
    _id *= BTC_option_chain_dict[date].ttms * 365 >= 7
    targeted_ids = BTC_option_chain_dict[date].ids[_id]
    option_ids_i = [i for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
    option_ids   = [_id for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
    option_slice = slice_option_chain(BTC_option_chain_dict[date], option_ids_i)
    
    # Take only the closest 
    ids_i_1m = np.argmin(np.abs(np.array(option_slice.ttms) * 365 - 30))
    option_slice = slice_option_chain(option_slice, [ids_i_1m])

    new_option_chain = smooth_option_chain(option_slice)

    results = []
    MAPEs = []
    for chi0 in chi0_arr:
        chi_p0, chi_m0 = chi0
        measure_change_params, measure_change_results = pricer.calibrate_measure_change_params_to_chain(option_chain=new_option_chain,
                                                                                                        params0 = params,
                                                                                                        bounds  = bounds,
                                                                                                        method  = method,
                                                                                                        errors_weights = errors_weights, 
                                                                                                        options = options,
                                                                                                        is_vega_weighted = True, 
                                                                                                        p0      = (sigma, chi_p0, chi_m0),
                                                                                                        verbose = True)
        
        _bid_ivs = np.concatenate(option_slice.bid_ivs)
        _ask_ivs = np.concatenate(option_slice.ask_ivs)
        _mid_ivs = (_bid_ivs + _ask_ivs) / 2

        model_ivs = pricer.compute_model_ivols_for_chain(option_chain=option_slice, params=measure_change_results)
        _model_ivs = np.concatenate(model_ivs)
        MAPE = np.mean( np.abs(_model_ivs - _mid_ivs)/_mid_ivs )
        results.append((measure_change_params, measure_change_results))
        MAPEs.append(MAPE)
        
        
    measure_change_params, measure_change_results = results[np.argmin(MAPEs)]
    MAPE = np.min(MAPEs)
    
    result = {'measure_change_params':measure_change_params, 'measure_change_results': measure_change_results,
              'MAPE':MAPE, 'n_options':len(_mid_ivs), 'is_pos_jump':is_pos_jump, 'is_neg_jump':is_neg_jump, 'option_ids':option_slice.ids}
    print(date, measure_change_params, MAPE, len(_mid_ivs), is_pos_jump, is_neg_jump, option_slice.ids)
    Q_results_1m[date] = result

----------- 2020-01-01 -----------
[ 0.2 -2.  -4. ] 0.0019870469689208387 6.000000000000001e-08
[ 0.81400426 -2.         -4.        ] 0.01132303262269145 6.000000000000001e-08
[ 1.57013519 -2.         -4.        ] 0.02752609978348606 6.000000000000001e-08
[ 0.5201401 -2.        -4.       ] 0.006020550109722746 6.000000000000001e-08
[ 0.27560636 -2.         -4.        ] 0.0026914955214032443 6.000000000000001e-08
[ 0.36649651 -2.         -4.        ] 0.0037655428769950913 6.000000000000001e-08
[ 0.21614763 -2.         -4.        ] 0.0021214885008314665 6.000000000000001e-08
[ 0.1794837 -2.        -4.       ] 0.0018295715805147544 6.000000000000001e-08
[ 0.15680283 -2.         -4.        ] 0.0016734749110835084 6.000000000000001e-08
[ 0.14276455 -2.         -4.        ] 0.0015866288652016983 6.000000000000001e-08
[ 0.13407751 -2.         -4.        ] 0.0015367166464446448 6.000000000000001e-08
[ 0.12870375 -2.         -4.        ] 0.0015073262779977709 6.000000000000001e-08
[ 0.12538055 

In [46]:
with open('Q_results%i.pickle'%year, 'wb') as f:
    pickle.dump(Q_results_1m, f)